<a href="https://colab.research.google.com/github/AnnaFattakhova/poetry-llm-additional/blob/main/counting_syllables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Устанавливаем Groq

!pip install groq
from groq import Groq

api_key = "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"
client = Groq(api_key=api_key)

Промт 1 (сhain-of-thought)

In [ ]:
# "deepseek-r1-distill-llama-70b",
def llm_inference(extracted_sample, user_topic):

    output = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "system", "content": """
                                          Генерировать стихотворения по заданным условиям:
                                          - в процессе генерации сегментируй текст по слогам
                                          - используй сегментацию для выбора рифмы
                                          - выводи результат в JSON формате
                                          - используй технику chain-of-thought с учетом слогового деления
                                          - считай количество гласных
                                          - если в запросе хайку, пересчитай количество слогов для валидации ответа
                                          """
            },
            {"role": "user",
             "content": f"сгенерируй стихотворение в жанре {extracted_sample}, тема стихотворения: {user_topic}"},
        ],
        response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "properties": {
                    "chain-of-thought": {"type": "string"},
                    "результат подсчета гласных": {"type": "string"},
                    "результат сегментации на слоги": {"type": "string"},
                    "сгенерированный стих": {"type": "string"},
                },
                "required": [
                    "chain-of-thought",
                    "результат подсчета гласных",
                    "результат сегментации на слоги",
                    "сгенерированный стих"
                ]
            }
        },
        stream=False,
        max_tokens=1024,
        temperature=0.7,
        top_p=0.1
    )

    return output.choices[0].message.content

In [ ]:
llm_inference('хайку', 'лес')

'{\n  "poem": [\n    "Солнце высоко в кронах",\n    "Шелест листьев в воздухе",\n    "Тени глубокие стоят"\n  ],\n  "syllable_count": {\n    "first_line": 5,\n    "second_line": 7,\n    "third_line": 5\n  },\n  "vowel_count": {\n    "first_line": 5,\n    "second_line": 5,\n    "third_line": 5\n  },\n  "validation": {\n    "haiku_valid": true,\n    "message": "Стихотворение соответствует структуре хайку (5-7-5) и теме \'лес\'."\n  }\n}'

Промт 2 (список гласных)

In [ ]:
def llm_inference(extracted_sample, user_topic):

    output = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "system", "content": """
                                          Генерировать стихотворения по заданным условиям:
                                          1. выведи правила для написания стихотворения в заданном жанре
                                          2. в процессе генерации сегментируй текст по слогам
                                          - используй сегментацию для выбора рифмы
                                          - выводи результат на основе техники chain-of-thought с учетом слогового деления
                                          - считай количество гласных для определения количества слогов (1 слог = 1 гласная)
                                          - список гласных: [а, о, е, ё, и, у, ы, э, ю, я]
                                          3. если в запросе хайку, пересчитай количество слогов для валидации ответа
                                          """
            },
            {"role": "user",
             "content": f"Сгенерируй стихотворение в жанре {extracted_sample} на тему '{user_topic}'"},
        ],


        stream=False,
        max_tokens=2000,
        temperature=0.7,
        top_p=0.1
    )

    return output.choices[0].message.content

In [ ]:
llm_inference('хайку', 'лес')

'<think>\nХорошо, мне нужно сгенерировать стихотворение в жанре хайку на тему "лес". Сначала я подумаю о правилах хайку. Хайку — это традиционная японская поэзия, состоящая из трёх строк с схемой слогов 5-7-5. Каждая строка должна содержать определённое количество слогов: первая и третья строки по 5 слогов, вторая — 7.\n\nТеперь я подумаю о теме "лес". Лес вызывает множество образов: деревья, листья, ветер, тишина, птицы и т.д. Я постараюсь передать спокойствие и гармонию леса.\n\nДалее, я должен сегментировать текст по слогам. Для этого я рассмотрю каждое слово и разделю его на слоги. Важно помнить, что в русском языке слог часто соответствует одной гласной. Список гласных: а, о, е, ё, и, у, ы, э, ю, я. Каждая гласная обычно соответствует одному слогу.\n\nНачну с первой строки. Мне нужно 5 слогов. Допустим, я беру слово "тишина". Разделив на слоги: ти-ши-на — это 3 слога. Затем добавлю "лес" — 1 слог. Но мне нужно 5, поэтому, возможно, я должен подобрать другое начало или добавить сло

Промт 3 (схема)

In [11]:
def llm_inference(extracted_sample, user_topic):

    output = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "system", "content": """
                                          Генерировать стихотворения по заданным условиям:
                                          1. выведи правила для написания стихотворения в заданном жанре
                                          2. создай канву, рамку для генерации стихотворения
                                          пример канвы для хайку:
                                          _ _ _ _ _ (5 слогов) // _ _ _ _ _ _ _ (7 слогов) // _ _ _ _ _ (5 слогов)
                                          3. в процессе генерации сегментируй текст по слогам
                                          - используй сегментацию для выбора рифмы
                                          - выводи результат на основе техники chain-of-thought с учетом слогового деления
                                          - считай количество гласных для определения количества слогов (1 слог = 1 гласная)
                                          - список гласных: [а, о, е, ё, и, у, ы, э, ю, я]
                                          4. если в запросе хайку, пересчитай количество слогов для валидации ответа
                                          5. Дай ответ в формате JSON
                                          """
            },
            {"role": "user",
             "content": f"Сгенерируй стихотворение в жанре {extracted_sample} на тему '{user_topic}'"},
        ],

        stream=False,
        max_tokens=2000,
        temperature=0.7,
        top_p=0.1
    )

    return output.choices[0].message.content

In [12]:
llm_inference('хайку', 'лес')

'<think>\nХорошо, мне нужно помочь пользователю сгенерировать стихотворение в жанре хайку на тему "лес". Сначала я должен понять, что такое хайку и каковы его основные правила. Хайку — это традиционная японская поэзия, состоящая из трёх строк с определённым количеством слогов: 5, 7, 5. \n\nПервым шагом я подумаю о теме "лес". Мне нужно выбрать элементы, которые ассоциируются с лесом: деревья, листья, ветер, тишина, возможно, птицы или солнечный свет, пробивающийся сквозь кроны. \n\nТеперь перейду к правилам написания хайку. Важно, чтобы каждая строка соответствовала количеству слогов: первая и третья строки по 5 слогов, вторая — 7. Также важно, чтобы стихотворение передавало определённое настроение или изображение.\n\nДалее, создам канву для хайку. Она будет выглядеть так:\n_ _ _ _ _ (5 слогов)\n_ _ _ _ _ _ _ (7 слогов)\n_ _ _ _ _ (5 слогов)\n\nТеперь приступлю к генерации стихотворения. Начну с первой строки. Подумаю о словах, которые описывают лес и соответствуют 5 слогам. Например, 

Промт 4 (rule-based)

In [13]:
def llm_inference(sample):

    output = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "system", "content": """
                                          **Правила деления на слоги**: В русском языке количество слогов равно количеству гласных в слове.
                                            Гласные: {а, е, ё, и, о, у, ы, э, ю, я}
                                            Минимальная единица слога: Один слог содержит хотя бы одну гласную.
                                            Границы слогов:

                                                Слог начинается с гласной, если перед ней нет другой гласной.
                                                Согласные между двумя гласными обычно относятся ко второй гласной (V-CV).
                                                Комбинации согласных разбиваются по фонетическим правилам:
                                                    С-С — первая согласная остаётся в предыдущем слоге, вторая идёт в новый.
                                                    С-С-С — если первая согласная сонорная ({л, м, н, р}), она может оставаться в предыдущем слоге (CС-СV), иначе — C-ССV.
                                                    С-С-С-С — чаще делится как CС-ССV, но возможны исключения.
                                            **Задача**:
                                            1. Создать мэппинг текста на слоги
                                            2. Создать список слогов для данного текста
                                            3. Сложить и вывести общее количество слогов в тексте.
                                          """
            },
            {"role": "user",
              "content": f"Подели текст на слоги: {sample}"},
        ],
        stream=False,
        max_tokens=2000,
        temperature=0.7,
        top_p=0.1
    )

    return output.choices[0].message.content

In [15]:
result = llm_inference('Среди высоких деревьев')
result

'<think>\nХорошо, мне нужно поделить текст "Среди высоких деревьев" на слоги по правилам русского языка. Сначала я вспомню, что количество слогов равно количеству гласных в слове. Гласные буквы: а, е, ё, и, о, у, ы, э, ю, я.\n\nНачну с разбора каждого слова по отдельности.\n\nПервое слово: "Среди". Посмотрю на гласные. "С" - согласная, "р" - согласная, "е" - гласная, "д" - согласная. Значит, здесь одна гласная "е", значит, один слог. Но подожди, может быть, я ошибся. "Среди" произносится как "среди", и обычно это один слог. Да, правильно.\n\nВторое слово: "высоких". Гласные: "ы", "о", "и". Три гласные, значит, три слога. Нужно определить, как их разделить. "Вы-со-ких". Да, так правильно.\n\nТретье слово: "деревьев". Гласные: "е", "е", "ев" (но "ев" считается как одна гласная). Значит, два слога. "Де-ревьев". Или "Де-ре-вьев"? Нет, правильно "Де-ревьев", потому что "р" и "в" между двумя гласными относятся ко второй гласной.\n\nТеперь объединяю все слоги: "Среди" (1) + "вы-со-ких" (3) + 

In [14]:
result = llm_inference('Шуршат листья на ветру')
result

'<think>\nХорошо, мне нужно поделить текст "Шуршат листья на ветру" на слоги по правилам русского языка. Сначала я вспомню, что количество слогов равно количеству гласных в слове. Гласные буквы: а, е, ё, и, о, у, ы, э, ю, я.\n\nНачну с первого слова "Шуршат". Посмотрю на гласные: у и а. Значит, два слога. Согласные между гласными: р и ш. Согласно правилам, они относятся ко второй гласной. Так что "Шу" и "ршат" не подходят, потому что "ршат" содержит гласную "а". Возможно, правильнее разделить как "Шур" и "шат". Но тогда "Шур" заканчивается на "р", а "шат" начинается с "ш". Это может быть правильно.\n\nТеперь второе слово "листья". Гласные: и и я. Два слога. Между ними согласные:сть. Согласно правилам, согласные между гласными относятся ко второй. Так что "ли" и "стья". Но "стья" начинается с "сть", что может быть проблемой. Возможно, правильнее разделить как "ли" и "стья".\n\nТретье слово "на". Гласные: а. Один слог.\n\nЧетвертое слово "ветру". Гласные: е и у. Два слога. Между ними сог

Промт 5 (promt generation + few-shot learning + подсчет гласных букв + chain-of-thought)

In [16]:
import re

def syllables(word):
    """Функция для подсчета слогов в русском слове"""
    vowels = "аеёиоуыэюяАЕЁИОУЫЭЮЯ"
    word = word.lower()
    return sum(1 for char in word if char in vowels)

def llm_inference(sample, n):
    # Получаем информацию о слогах
    syllable_count = syllables(sample)
    info = f"Слово '{sample}' содержит {syllable_count} слогов."

    output = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "system", "content": """
                                          **Правила деления на слоги**: В русском языке количество слогов равно количеству гласных в слове.
                                            Гласные: {а, е, ё, и, о, у, ы, э, ю, я}

                                              #### **1. Разбиение слова на фонемные группы**
                                              Используем регулярные выражения для выделения слоговых единиц:
                                              - `V+` — последовательность гласных (центр слога)
                                              - `С+` — последовательность согласных (разделители слогов)

                                              #### **2. Построение структуры слова**
                                              Пример разбиения:
                                              - `де-ре-вьев` (3 слога)
                                              - `сре-ди` (2 слога)
                                              - `счаст-лив` (2 слога)
                                              - `е-го` (2 слога)
                                              - `в са-ду` (2 слога)


                                              #### **3. Алгоритм разбиения (Pseudocode)**

                                              import re

                                              def split_syllables(word):
                                                  vowels = "аеёиоуыэюя"
                                                  pattern = re.compile(f"([{vowels}]+[^ {vowels}]*)", re.IGNORECASE)
                                                  return "-".join(pattern.findall(word))

                                              print(split_syllables("деревьев"))  # де-ре-вьев
                                              print(split_syllables("счастлив"))  # счаст-лив
                                              print(split_syllables("его"))  # е-го

                                             **Задача**:
                                            1. Написать строку текста с использованием данного слова
                                            2. Посчитать количество слогов
                                            3. Если количество слогов соответствует заданному, показать результат, иначе повторить пункты 1-2
                                            """
              },
              {"role": "user",
              "content": f"напиши строку из {n} слогов со словом: {sample}. {info}"},
          ],
        stream=False,
        max_tokens=2000,
        temperature=0.7,
        top_p=0.1
    )
    return output.choices[0].message.content

In [17]:
result = llm_inference('природа', 7)
result

'<think>\nХорошо, мне нужно написать строку из 7 слогов, используя слово "природа", которое само по себе имеет 3 слога. Сначала я подумаю, как разбить слово "природа" на слоги. Поскольку в нем три гласные (и, о, а), то слоги будут при-ро-да. Теперь мне нужно создать предложение, где общее количество слогов в строке будет 7, включая эти три от "природа".\n\nЯ начну с того, что подумаю, какие слова можно добавить до или после "природа", чтобы общее количество слогов составило 7. Например, можно использовать глаголы или прилагательные, которые добавляют слоги. Допустим, я беру слово "наслаждаться", которое имеет 4 слога (на-сла-жа-ться). Тогда, если я напишу "Наслаждаюсь природой", то общее количество слогов будет 3 (природа) + 4 (наслаждаться) = 7. Проверю: при-ро-да (3) + на-сла-жа-ться (4) = 7. Да, это подходит.\n\nТеперь проверю, правильно ли я разбил слова на слоги. "Наслаждаюсь" разбивается на на-сла-жа-юсь, что действительно 4 слога. "Природа" как при-ро-да, 3 слога. Вместе они сос

In [19]:
result = llm_inference('луна', 8)
result

'<think>\nХорошо, мне нужно написать строку из 8 слогов, используя слово "луна", которое само по себе имеет 2 слога. Сначала я подумаю, как создать предложение, где общее количество слогов будет 8, включая те 2, что в "луна".\n\nНачну с того, что подберу другие слова, которые имеют определённое количество слогов. Например, "на" – 1 слог, "небо" – 2 слога, "сияет" – 2 слога, "в" – 1 слог, "ночь" – 1 слог. Если я сложу их вместе, получится: "на небо луна сияет в ночь". Давайте посчитаем слоги:\n\n- "на" – 1\n- "небо" – 2\n- "луна" – 2\n- "сияет" – 2\n- "в" – 1\n- "ночь" – 1\n\nВсего: 1+2+2+2+1+1 = 9 слогов. Ой, слишком много. Нужно уменьшить на 1 слог.\n\nМожет, убрать слово "в"? Тогда предложение станет: "на небо луна сияет ночь". Посчитаем:\n\n- "на" – 1\n- "небо" – 2\n- "луна" – 2\n- "сияет" – 2\n- "ночь" – 1\n\nТеперь 1+2+2+2+1 = 8 слогов. Отлично! Проверю, чтобы предложение было осмысленным и красивым. "На небо луна сияет ночь" звучит хорошо и соответствует требованиям.\n</think>\n\

In [20]:
result = llm_inference('ночь', 10)
result

'<think>\nХорошо, мне нужно написать строку из 10 слогов, используя слово "ночь", которое само по себе имеет 1 слог. Для начала, я подумаю, как можно расширить предложение, добавив другие слова, чтобы общее количество слогов достигло 10.\n\nНачну с того, что подумаю о других словах, которые легко сочетаются с "ночь". Может быть, описать ночь, добавить некоторые действия или объекты, связанные с ночью. Например, можно написать о том, что происходит ночью, какие звуки или зрелища есть.\n\nДопустим, я начну с предложения: "Ночью было тихо." Это предложение содержит 5 слогов: Но-чи – 2 слога, бы-ло – 2 слога, ти-хо – 2 слога. В сумме 2+2+2=6 слогов. Это еще не достаточно.\n\nТогда я подумаю, как можно добавить еще несколько слов. Может быть, описать, что делает кто-то ночью. Например, "Ночью было тихо, и я сидел у окна." Теперь посчитаю слоги: "Но-чи" – 2, "бы-ло" – 2, "ти-хо" – 2, "и" – 1, "я" – 1, "си-дель" – 2, "у" – 1, "о-кна" – 2. В сумме 2+2+2+1+1+2+1+2=12 слогов. Это уже больше 10.\

In [21]:
result = llm_inference('осенние', 6)
result

'<think>\nХорошо, я получил задание написать строку из 6 слогов, используя слово "осенние", которое само по себе содержит 4 слога. Мне нужно убедиться, что в итоговой строке будет именно 6 слогов.\n\nПервым делом, я разбил слово "осенние" на слоги. Как упоминалось, это де-ре-вьев, то есть 3 слога. Но в данном случае, слово "осенние" имеет 4 слога: о-сен-ни-е. Проверю это, посчитав гласные: о, е, и, е – четыре гласные, значит, четыре слога.\n\nТеперь мне нужно создать предложение, где общее количество слогов будет 6. Поскольку "осенние" уже занимает 4 слога, мне нужно добавить слова, которые в сумме дадут еще 2 слога. Я подумаю, какие слова можно использовать. Возможно, я возьму краткие слова, такие как "и" (1 слог), "как" (1 слог), "листья" (2 слога), "ветер" (2 слога) и т.д.\n\nПопробую составить предложение: "Осенние листья падают." Давайте посчитаем слоги:\n- Осенние – 4 слога\n- листья – 2 слога\n- падают – 3 слога\nВсего: 4 + 2 + 3 = 9 слогов. Это слишком много.\n\nНужно уменьшить

In [22]:
result = llm_inference('играет', 9)
result

'<think>\nХорошо, мне нужно написать строку из 9 слогов, используя слово "играет", которое само по себе содержит 3 слога. Сначала я подумаю, как определить количество слогов в других словах. В русском языке количество слогов соответствует количеству гласных в слове. Гласные буквы: а, е, ё, и, о, у, ы, э, ю, я.\n\nИтак, я начну с разбиения слова "играет" на слоги. "Играет" имеет гласные И, А, Е, то есть 3 слога. Теперь мне нужно найти другие слова, которые в сумме с "играет" дадут 9 слогов. Поскольку "играет" уже дает 3, мне нужно еще 6 слогов.\n\nЯ подумаю о других словах с разным количеством слогов. Например, "мальчик" имеет 2 слога (А и И), "бегает" имеет 3 слога (Е, А, Е). Но мне нужно 6 слогов, поэтому, возможно, я могу использовать несколько слов.\n\nДопустим, я беру "мальчик" (2 слога) и "бегает" (3 слога). Вместе с "играет" это дает 2 + 3 + 3 = 8 слогов, что не хватает. Тогда я могу добавить еще одно слово, например, "собака" (3 слога). Теперь общее количество слогов будет 2 + 3

In [23]:
result = llm_inference('трель', 4)
result

'<think>\nХорошо, мне нужно написать строку из четырёх слогов, используя слово "трель", которое само по себе имеет один слог. Для начала, я подумаю, как можно расширить предложение, добавив слова, которые добавят слоги.\n\nНапример, я могу начать с глагола, который имеет несколько слогов. Допустим, "поёт" – это два слога. Затем, добавить прилагательное, например, "красивая" – три слога. Далее, добавить существительное, например, "песня" – два слога. Наконец, использовать слово "трель" – один слог.\n\nСложив всё вместе, получится предложение: "Поёт красивая песня трель." Давайте посчитаем слоги:\n\n- Поёт – 2 слога\n- красивая – 3 слога\n- песня – 2 слога\n- трель – 1 слог\n\nВсего: 2 + 3 + 2 + 1 = 8 слогов. Это больше четырёх, так что нужно сократить.\n\nПопробую упростить. Вместо "поёт красивая песня", можно использовать "поёт трель". Это будет:\n\n- Поёт – 2 слога\n- трель – 1 слог\n\nВсего: 3 слога. Недостаточно. Добавлю слово "красивая" перед "трель": "Поёт красивая трель."\n\n- По